# Buscador de compañero piso

## 1. SETUP

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

## 2. CARGA DE DATOS

In [2]:
# 2. CARGA DE DATOS
df = pd.read_csv('C:\\Users\\Kim\\Downloads\\Habitaciones\\Habitaciones\\dataset_inquilinos.csv',index_col = 'id_inquilino')
df.columns = [
'horario', 'bioritmo', 'nivel_educativo', 'leer', 'animacion', 
'cine', 'mascotas', 'cocinar', 'deporte', 'dieta', 'fumador',
'visitas', 'orden', 'musica_tipo', 'musica_alta', 'plan_perfecto', 'instrumento'
]
df.head()

,horario,bioritmo,nivel_educativo,leer,animacion,cine,mascotas,cocinar,deporte,dieta,fumador,visitas,orden,musica_tipo,musica_alta,plan_perfecto,instrumento
id_inquilino,,,,,,,,,,,,,,,,,
1,noche,nocturno,secundaria,si,no,no,con mascotas,pedir comida,si,si,si,si,relajada,reggaeton,no,casa,si
2,noche,nocturno,secundaria,no,no,no,con mascotas,pedir comida,no,no,si,si,relajada,reggaeton,si,casa,si
3,noche,madrugador,universitaria,no,no,si,con mascotas,pedir comida,no,no,si,no,ordenada,pop,no,casa,no
4,noche,nocturno,universitaria,no,si,no,con mascotas,pedir comida,si,no,no,si,relajada,rock,si,casa,si
5,mañana,nocturno,universitaria,si,no,no,sin mascotas,cocinar,no,no,no,no,relajada,pop,no,casa,no


## 3. ONE HOT ENCODING

In [3]:
#Datos sobre mí
valores = ['mañana', 'madrugador', 'universitaria', 'si', 'si', 'si', 'sin mascotas', 'cocinar',
           'si', 'si', 'no', 'si', 'ordenada', 'clasica', 'no', 'salir', 'no']

# Columnas para el DataFrame
columnas = [
    'horario', 'bioritmo', 'nivel_educativo', 'leer', 'animacion', 
    'cine', 'mascotas', 'cocinar', 'deporte', 'dieta', 
    'fumador', 'visitas', 'orden', 'musica_tipo', 'musica_alta', 
    'plan_perfecto', 'instrumento'
]

# Crear el DataFrame
sobre_mi = pd.DataFrame([valores], columns=columnas)

# Mostrar el DataFrame
sobre_mi.head()

,horario,bioritmo,nivel_educativo,leer,animacion,cine,mascotas,cocinar,deporte,dieta,fumador,visitas,orden,musica_tipo,musica_alta,plan_perfecto,instrumento
0,mañana,madrugador,universitaria,si,si,si,sin mascotas,cocinar,si,si,no,si,ordenada,clasica,no,salir,no


In [4]:
encoder = OneHotEncoder(sparse=False)
# Realizar el one-hot encoding
df_encoded = encoder.fit_transform(df)
sobre_mi_encoded = encoder.transform(sobre_mi)
"""
chicos = pd.DataFrame(df_encoded, columns=encoder.get_feature_names_out())
chicos.head(1)
yo = pd.DataFrame(nuevo_inquilino_encoded, columns=encoder.get_feature_names_out())
yo
"""

C:\Users\Kim\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


'\nchicos = pd.DataFrame(df_encoded, columns=encoder.get_feature_names_out())\nchicos.head(1)\nyo = pd.DataFrame(nuevo_inquilino_encoded, columns=encoder.get_feature_names_out())\nyo\n'

In [5]:
sobre_mi_encoded

array([[1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
        0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 1., 0.]])

In [6]:
sobre_mi_encoded.shape

(1, 37)

## 4. MATRIZ DE SIMILIARIDAD

In [7]:
# Definir los pesos para cada columna
pesos = {
    'horario': 1.0,
    'bioritmo': 1.0,
    'nivel_educativo': 2.0,
    'leer': 1.0,
    'animacion': 1.0,
    'cine': 1.0,
    'mascotas': 1.0,
    'cocinar': 1.0,
    'deporte': 2.0,
    'dieta': 1.0,
    'fumador': 1.5,
    'visitas': 1.0,
    'orden': 1.0,
    'musica_tipo': 1.0,
    'musica_alta': 1.0,
    'plan_perfecto': 1.0,
    'instrumento': 1.0
}


In [8]:
# Aplicar los pesos a las columnas codificadas
for col, peso in pesos.items():
    mask = [feature.startswith(col) for feature in encoder.get_feature_names_out()]
    df_encoded[:, mask] *= peso
    sobre_mi_encoded[:, mask] *= peso


In [21]:
sobre_mi_encoded

array([[1. , 0. , 1. , 0. , 0. , 0. , 2. , 0. , 1. , 0. , 1. , 0. , 1. ,
        0. , 1. , 1. , 0. , 0. , 2. , 0. , 1. , 1.5, 0. , 0. , 1. , 1. ,
        0. , 1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. , 1. , 0. ]])

In [10]:
# Calcular la matriz de similaridad utilizando el punto producto
matriz_s = np.dot(df_encoded, sobre_mi_encoded.T)
matriz_s

array([[ 8.  ],
       [ 1.  ],
       [ 9.  ],
       ...,
       [ 7.  ],
       [14.25],
       [ 4.  ]])

In [11]:
# Define el rango de destino
rango_min = -100
rango_max = 100

In [12]:
# Encontrar el mínimo y máximo valor en matriz_s
min_original = np.min(matriz_s)
max_original = np.max(matriz_s)

In [13]:
print("min_original: " + str(min_original))
print("max_original: " + str(max_original))

min_original: 1.0
max_original: 22.25


In [14]:
# Reescalar la matriz
matriz_s_reescalada = ((matriz_s - min_original) / (max_original - min_original)) * (rango_max - rango_min) + rango_min

In [15]:
matriz_s_reescalada

array([[ -34.11764706],
       [-100.        ],
       [ -24.70588235],
       ...,
       [ -43.52941176],
       [  24.70588235],
       [ -71.76470588]])

In [16]:
 df.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            11991, 11992, 11993, 11994, 11995, 11996, 11997, 11998, 11999,
            12000],
           dtype='int64', name='id_inquilino', length=12000)

In [17]:
# Pasar a Pandas
df_similaridad = pd.DataFrame(matriz_s_reescalada, columns = ['Similitud'], index = df.index)
df_similaridad

,Similitud
id_inquilino,
1,-34.117647
2,-100.000000
3,-24.705882
4,5.882353
5,5.882353
...,...
11996,-43.529412
11997,-43.529412
11998,-43.529412


In [18]:
top = df_similaridad['Similitud'].sort_values(ascending=False).head(3)
top

id_inquilino
2610    100.0
3085    100.0
7726    100.0
Name: Similitud, dtype: float64

## Resultado

In [19]:
compatibles = df.loc[top.index]
compatibles

,horario,bioritmo,nivel_educativo,leer,animacion,cine,mascotas,cocinar,deporte,dieta,fumador,visitas,orden,musica_tipo,musica_alta,plan_perfecto,instrumento
id_inquilino,,,,,,,,,,,,,,,,,
2610,mañana,madrugador,universitaria,si,si,si,sin mascotas,cocinar,si,no,no,si,ordenada,pop,no,salir,no
3085,mañana,madrugador,universitaria,si,si,si,sin mascotas,cocinar,si,no,no,si,ordenada,clasica,no,casa,no
7726,mañana,madrugador,universitaria,si,no,si,sin mascotas,cocinar,si,si,no,si,ordenada,rock,no,salir,no


In [20]:
sobre_mi

,horario,bioritmo,nivel_educativo,leer,animacion,cine,mascotas,cocinar,deporte,dieta,fumador,visitas,orden,musica_tipo,musica_alta,plan_perfecto,instrumento
0,mañana,madrugador,universitaria,si,si,si,sin mascotas,cocinar,si,si,no,si,ordenada,clasica,no,salir,no
